In [8]:
### Grading script code 
### You don't need to read this, proceed to the next cell
import sys
import functools
ipython = get_ipython()

def set_traceback(val):
    method_name = "showtraceback"
    setattr(
        ipython,
        method_name,
        functools.partial(
            getattr(ipython, method_name),
            exception_only=(not val)
        )
    )

class AnswerError(Exception):
  def __init__(self, message):
    pass

def exec_test(f, question):
    try:
        f()
        print(question + " Pass")
    except:
        set_traceback(False) # do not remove
        raise AnswerError(question + " Fail")

# Week 5 Problem Set

## Homework

**HW1.** *Dictionary:* Write two functions:
1. `count_degrees(G)`: which sums up the degrees of all vertices in the graph. The degree of a vertex is defined as the number of edges connected to a Vertex. The argument `G` is a dictionary that represents the graph.
2. `count_edges(G)`: which counts the number of edges in the graph. An edge is defined as a connection between two vertices. The argument `G` is a dictionary.

In [9]:
def count_degrees(G):
    ###
    ### YOUR CODE HERE
    ###
    pass

def count_edges(G):
    ###
    ### YOUR CODE HERE
    ###
    pass

In [10]:
g1 = {"A": ["B", "E"],
      "B": ["A", "C"],
      "C": ["B", "D", "E"],
      "D": ["C"],
      "E": ["A", "C"]}

d = count_degrees(g1)
e = count_edges(g1)

assert d == 10
assert e == 5

In [11]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW2.** Create a class called `GraphSearch` which is a subclass of the class `Graph`. This class should override the method `_create_vertex(id)` and instantiate a `VertexSearch` object instead of `Vertex`.

In [12]:
# Copy Vertex class from Cohort
class Vertex:
    pass
    ###
    ### YOUR CODE HERE
    ###


In [13]:
# Copy VertexSearch class from Cohort

import sys

class VertexSearch(Vertex):
    def __init__(self, id_=""):
        ###
        ### YOUR CODE HERE
        ###


In [14]:
#Copy Graph over from Cohort
class Graph:
    def __init__(self):
        self.vertices = {}
    ###
    ### YOUR CODE HERE
    ###


In [15]:
class GraphSearch(Graph):
    ##BEGIN SOLUTION
    def _create_vertex(self, id_):
        return VertexSearch(id_)
    ##END SOLUTION
    pass

In [16]:
g2 = GraphSearch()
g2.add_vertex("Z")
assert(type(g2.vertices["Z"]) == type(VertexSearch()))

In [17]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW3.** *Undirected Graph:* **You need to complete CS5 and CS6 before you do this homework.** Paste your answer for `Search2D` and `SearchBFS` classes into the cell below.

Create a subclass of `GraphSearch` class called `UGraphSearch` for undirected graphs. All edges in undirected graphs are bidirectional (i.e. vertex1 <-> vertex2). 
This means that you need to override the `add_edge()` method.

In [18]:
#Copy Queue over from Wk4 Homework
class Queue:
    pass
    ###
    ### YOUR CODE HERE
    ###


In [19]:
# copy Search2D over from CS5

import sys

class Search2D:
    def __init__(self, g):
        self.graph = g
        
    ###
    ### YOUR CODE HERE
    ###


In [20]:
# copy SearchBFS over from CS6

class SearchBFS(Search2D):
    pass
    ###
    ### YOUR CODE HERE
    ###


In [21]:
class UGraphSearch(GraphSearch):
    def add_edge(self, start_v, end_v, weight=0):
        ###
        ### YOUR CODE HERE
        ###
        pass

In [22]:
g2 = UGraphSearch()
assert g2.vertices == {} and g2.num_vertices == 0
g2.add_vertex("L")
g2.add_vertex("I")
g2.add_vertex("S")
g2.add_vertex("P")
assert g2.num_vertices == 4
assert "L" in g2
assert "I" in g2
assert "S" in g2
assert "P" in g2
g2.add_edge("L", "I")
g2.add_edge("I", "S")
g2.add_edge("S", "P")
assert sorted(g2.get_neighbours("L")) == ["I"]
assert sorted(g2.get_neighbours("I")) == ["L", "S"]
assert sorted(g2.get_neighbours("S")) == ["I", "P"]
assert sorted(g2.get_neighbours("P")) == ["S"]

In [23]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [24]:
g2 = UGraphSearch()
g2.add_vertex("r")
g2.add_vertex("s")
g2.add_vertex("t")
g2.add_vertex("u")
g2.add_vertex("v")
g2.add_vertex("w")
g2.add_vertex("x")
g2.add_vertex("y")
g2.add_vertex("z")
g2.add_edge("r", "s")
g2.add_edge("r", "v")
g2.add_edge("s", "w")
g2.add_edge("t", "u")
g2.add_edge("t", "x")
g2.add_edge("t", "w")
g2.add_edge("u", "t")
g2.add_edge("u", "x")
g2.add_edge("u", "y")
g2.add_edge("v", "r")
g2.add_edge("w", "s")
g2.add_edge("w", "t")
g2.add_edge("w", "x")
g2.add_edge("x", "w")
g2.add_edge("x", "t")
g2.add_edge("x", "u")
g2.add_edge("x", "y")
g2.add_edge("y", "u")
g2.add_edge("y", "x")
gs = SearchBFS(g2)
gs.search_from("s")
assert gs.graph.get_vertex("s").d == 0
assert gs.graph.get_vertex("t").d == 2
assert gs.graph.get_vertex("u").d == 3
assert gs.graph.get_vertex("v").d == 2
assert gs.graph.get_vertex("w").d == 1
assert gs.graph.get_vertex("x").d == 2
assert gs.graph.get_vertex("y").d == 3
assert gs.graph.get_vertex("r").d == 1
ans = gs.get_shortest_path("s", "u")
assert ans == ["s", "w", "t", "u"] or ans == ["s", "w", "x", "u"]
ans = gs.get_shortest_path("v", "s")
assert ans == ["v", "r", "s"]
ans = gs.get_shortest_path("v", "y")
assert ans == ["v", "r", "s", "w", "x", "y"]

In [25]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW4.** *Depth-First-Search:* Create a class `SearchDFS` as a child class of `Search2D` to implement Depth-First-Search algorithm. You should add one additional attribute:
- `time`: which is an attribute to record the discovery time and the finishing time.

The class should also implement the following methods:
- `search()`: which modifies the vertices' properties such as `colour`, `d`, and `parent` following Depth-First-Search algorithm.
- `dfs_visit(vert)`: which is the recursive method for visiting a vertex in Depth-First-Search algorithm.

In [26]:
import sys

class SearchDFS(Search2D):
    def __init__(self, g):
        self.graph = g
        self.time = 0
      
    def search(self):
        ###
        ### YOUR CODE HERE
        ###
        pass
    
    def dfs_visit(self, vert):
        ###
        ### YOUR CODE HERE
        ###
        pass

In [27]:
g4 = GraphSearch()
g4.add_vertex("e")
g4.add_vertex("m")
g4.add_vertex("a")
g4.add_vertex("c")
g4.add_vertex("s")
g4.add_edge("e", "m")
g4.add_edge("m", "a")
g4.add_edge("a", "c")
g4.add_edge("c", "s")
gs = SearchDFS(g4)
gs.search()
assert gs.graph.get_vertex("e").parent == None 
assert gs.graph.get_vertex("m").parent == gs.graph.get_vertex("e")

assert gs.graph.get_vertex("m").d == 2 and gs.graph.get_vertex("a").f == 8
assert gs.graph.get_vertex("c").d == 4 and gs.graph.get_vertex("s").f == 6

In [28]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW5.** *Topological Sort:* Create a function that takes in a `SearchDFS` object to perform a topological sort:
- `topological_sort(g)`: which takes in a `SearchDFS` object and returns a list of `VertexSearch` objects sorted based on their `f` attribute. This method should call the `search()` method of the `SearchDFS` to first calculate the `f` attribute of the vertices. Note that you need to copy the `SearchDFS` object of your input argument so as not to mutate the object.

In [29]:
import sys
import copy

def topological_sort(g):
    ###
    ### YOUR CODE HERE
    ###
    pass

In [30]:
import copy
g = GraphSearch()
g.add_vertex("3/4 cup milk")
g.add_vertex("1 egg")
g.add_vertex("1 tbl oil")
g.add_vertex("1 cup mix")
g.add_vertex("heat syrup")
g.add_vertex("heat griddle")
g.add_vertex("pour 1/4 cup")
g.add_vertex("turn when bubbly")
g.add_vertex("eat")
g.add_edge("3/4 cup milk", "1 cup mix")
g.add_edge("1 egg", "1 cup mix")
g.add_edge("1 tbl oil", "1 cup mix")
g.add_edge("1 cup mix", "heat syrup")
g.add_edge("1 cup mix", "pour 1/4 cup")
g.add_edge("heat griddle", "pour 1/4 cup")
g.add_edge("pour 1/4 cup", "turn when bubbly")
g.add_edge("turn when bubbly", "eat")
g.add_edge("heat syrup", "eat")
gs = SearchDFS(g)  

path = topological_sort(gs)
ans = [v.f for v in copy.copy(path)]
assert ans == [18, 16, 14, 12, 11, 10, 9, 6, 5]
ans = [v.id_ for v in copy.copy(path)]
assert ans == ['heat griddle', '1 tbl oil', '1 egg', '3/4 cup milk', '1 cup mix', 'pour 1/4 cup', 'turn when bubbly', 'heat syrup', 'eat']

In [31]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
